In [2]:
import json

with open('tools/tools.json', encoding='utf-8', mode='r') as json_file:
    tools = json.load(json_file)

In [2]:
tools[1]

{'tool_name': 'enumerate_objects',
 'description': 'Detects objects in the environment and returns a list of recognized objects.',
 'parameters': {},
 'use_cases': [{'user_request': 'What objects are in the room?',
   'command': 'enumerate_objects()',
   'return_description': 'Returns a list of visible objects recognized by the assistant.'}]}

In [3]:
def generate_prompt_0(tool: dict):
   """Generates a prompt for a tool.

   Args:
       tool (dict): A dictionary containing the tool's information.
   """
   answer_format = {
        "user_request": "<the generated user request>",
        "command": "command_to_execute(parameters)"
      }
   answer_beginning = r'{"user_request": "'
   parameters = ', '.join([f'{parameter}: {" | ".join(types)}' for parameter, types in tool.get('parameters', {}).items()])
   tool_description = f"'{tool['tool_name']}({parameters})' is a tool that {tool['description']}"
   return f"You are a truthful assistant whose role is to generate 20 possible user requests for the following assistive tool:\n{tool_description}\nYou should use the following format for your answers: `{answer_format}`\nThe users are visually impaired and blind individuals.\nYour answer:\n[\n{answer_beginning}"

def generate_prompt_1(tool: dict):
   """Generates a prompt for a tool.

   Args:
       tool (dict): A dictionary containing the tool's information.
   """
   parameters = ', '.join([f'{parameter}: {" | ".join(types)}' for parameter, types in tool.get('parameters', {}).items()])
   tool_description = f"'{tool['tool_name']}({parameters})' is a tool that {tool['description']}"
   use_case = tool['use_cases'][0]['user_request']
   answer_beginning = r'{"user_request": "'
   generation_format = {
        "user_request": tool['use_cases'][0]['user_request'],
        "command": tool['use_cases'][0]['command']
      }
   return f"You are a truthful assistant whose role is to generate 20 possible user requests for the following assistive tool:\n{tool_description}\nOne possible user request is: `{use_case}`\nthe format of the generation should follow this:\n {generation_format}\nand you should generate other possible user requests.\nThe users are visually impaired and blind individuals.\nYour answer:\n[\n{answer_beginning}"

print(generate_prompt_0(tools[1]))
print(generate_prompt_1(tools[1]))

You are a trustful assistant whose role is to generate 20 possible user requests for the following assistive tool:
'enumerate_objects()' is a tool that Detects objects in the environment and returns a list of recognized objects.
You should use the following format for your answers: `{'user_request': '<the generated user request>', 'command': 'command_to_execute(parameters)'}`
The users are visually impaired and blind individuals.
Your answer:
[
{"user_request": "
You are a trustful assistant whose role is to generate possible user requests for the following assistive tool:
'enumerate_objects()' is a tool that Detects objects in the environment and returns a list of recognized objects.
One possible user request is: `What objects are in the room?`
the format of the generation should follow this:
 {'user_request': 'What objects are in the room?', 'command': 'enumerate_objects()'}
and you should generate other possible user requests.
The users are visually impaired and blind individuals.
Y

In [4]:
# Save the prompts for all the tools in 2 files (prompt_0.json and prompt_1.json)
with open('tools/prompts_0.json', encoding='utf-8', mode='w') as json_file:
    json.dump([generate_prompt_0(tool) for tool in tools], json_file, indent=2)

with open('tools/prompts_1.json', encoding='utf-8', mode='w') as json_file:
    json.dump([generate_prompt_1(tool) for tool in tools], json_file, indent=2)
